In [28]:
import sopa
import spatialdata
import pandas as pd
import scanpy as sc

In [29]:
Tissue = "Region2"
TableFolder = "REGION2_TABLES"

adata_phen = sc.read_h5ad(f"/Volumes/ProstateCancerEvoMain/dbs/Ongoing/{Tissue}/{TableFolder}/{Tissue}_Xenium_Phen_HE_Integrated.Protein_PhenCycTable.V1.h5ad")
adata_native = sc.read_h5ad(f"/Volumes/ProstateCancerEvoMain/dbs/Ongoing/{Tissue}/{TableFolder}/{Tissue}_Xenium_Phen_HE_Integrated.GeneTranscripts_XStock_Native.V1.h5ad")


sdata_file = f"/Volumes/ProstateCancerEvoMain/dbs/Ongoing/Region4/{Tissue}_Xenium_Phen_HE_Integrated.V2.ongoing.zarr"

channel_names = {
    '0':  'DAPI_PhenCyc', 
    '1':  'CD8_PhenCyc', 
    '2':  'CD31_PhenCyc', 
    '3':  'CD11c_PhenCyc', 
    '4':  'CD34_PhenCyc', 
    '5':  'CD20_PhenCyc', 
    '6':  'CD4_PhenCyc', 
    '7':  'CD21_PhenCyc', 
    '8':  'Ki67_PhenCyc', 
    '9':  'CD68_PhenCyc', 
    '10': 'E_cadherin_PhenCyc', 
    '11': 'pten_PhenCyc', 
    '12': 'NF-H_PhenCyc', 
    '13': 'FOXp3_PhenCyc', 
    '14': 'CD45_PhenCyc', 
    '15': 'TFAM_PhenCyc', 
    '16': 'PDL-1_PhenCyc', 
    '17': 'SOX2_PhenCyc', 
    '18': 'Vimentin_PhenCyc', 
    '19': 'HIF1A_PhenCyc', 
    '20': 'CD44_PhenCyc', 
    '21': 'Keratin_8_18_PhenCyc', 
    '22': 'aSMA_PhenCyc', 
    '23': 'EpCAM_PhenCyc', 
    '24': 'K14_PhenCyc'
}

#sdata = spatialdata.read_zarr(sdata_file)
#sdata

In [30]:
adata_phen.obs["cell_id"]

aaaaaaaa-1    aaaaealn-1
aaaaaaab-1    aaaaeaol-1
aaaaaaac-1    aaabgafi-1
aaaaaaad-1    aaabhken-1
aaaaaaae-1    aaabhmik-1
                 ...    
aaaejcpe-1    oilpooan-1
aaaejcpf-1    oimajkbj-1
aaaejcpg-1    oimalhbk-1
aaaejcph-1    oimaomgd-1
aaaejcpi-1    oimbdlnd-1
Name: cell_id, Length: 299769, dtype: object

In [14]:
#adata_stock = adata_native.copy()

# for table in ["Proteins_PhenoCycler_byXStock", "GeneTranscripts_XStockC"]:
#     sdata.tables[table].obs["cell_id"] = adata_stock.obs["cell_id"].to_list()
#     sdata.tables[table].obs["cell_id"].index = adata_stock.obs["cell_id"].index

#     for feat in ["region", "slide", "area"]:
#         sdata.tables[table].obs[feat].index = adata_stock.obs["cell_id"].to_list()

# Phenocycler Data will be introduced as obs column !

In [31]:
adata_b = adata_native.copy()
adata_a = adata_phen.copy()

#for idx in range(adata_b.obs.shape[1]):
#    adata_b.obs[str(idx)] = adata_a.X[:,idx]

import scipy.sparse as sp

# 2) Extract a dense array from adata_a.X if necessary
X = adata_a.X.toarray() if sp.issparse(adata_a.X) else adata_a.X

# 4) Loop over each column and assign it using the channel_names dict
for idx in range(X.shape[1]):
    col_name = channel_names[str(idx)]
    adata_b.obs[col_name] = X[:, idx]

In [16]:
adata = adata_b
adata

AnnData object with n_obs × n_vars = 299769 × 5101
    obs: 'cell_id', 'transcript_counts', 'control_probe_counts', 'genomic_control_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'nucleus_count', 'segmentation_method', 'region', 'z_level', 'DAPI_PhenCyc', 'CD8_PhenCyc', 'CD31_PhenCyc', 'CD11c_PhenCyc', 'CD34_PhenCyc', 'CD20_PhenCyc', 'CD4_PhenCyc', 'CD21_PhenCyc', 'Ki67_PhenCyc', 'CD68_PhenCyc', 'E_cadherin_PhenCyc', 'pten_PhenCyc', 'NF-H_PhenCyc', 'FOXp3_PhenCyc', 'CD45_PhenCyc', 'TFAM_PhenCyc', 'PDL-1_PhenCyc', 'SOX2_PhenCyc', 'Vimentin_PhenCyc', 'HIF1A_PhenCyc', 'CD44_PhenCyc', 'Keratin_8_18_PhenCyc', 'aSMA_PhenCyc', 'EpCAM_PhenCyc', 'K14_PhenCyc', 'area', 'centroid-0', 'centroid-1', 'centroid_local-1', 'centroid_weighted_local-0', 'centroid_weighted_local-1', 'euler_number', 'extent', 'feret_diameter_max', 'intensity_max', 'intensity_min', 'intensity_mean', 'solidity', 'eccentricity', 'pe

# After That Point the calculated skimage will be introduced as obs to anndata

In [32]:
#Read all of the properties CSVs and collect into one merged DataFrame
import pandas as pd

# 1) Read all your CSVs into a list of DataFrames
dfs = []
for i in range(0,2):
    print(i)
    path = f"{Tissue}_ExtractedProps_forChannel{i}.csv"
    # assume your files have a header row and the first column is an index
    df = pd.read_csv(path, index_col=0)
    dfs.append(df)

# 2) Make sure they all have the same shape, index & columns
assert all(df.shape == dfs[0].shape for df in dfs)
assert all((df.index == dfs[0].index).all()   for df in dfs)
assert all((df.columns == dfs[0].columns).all() for df in dfs)

# 3) Create the merged DataFrame of object‐dtype
merged = dfs[0].copy().astype(object)

# 4) Fill each cell with a list of values from each df
for idx in merged.index:
    for col in merged.columns:
        merged.at[idx, col] = [df.at[idx, col] for df in dfs]

0
1


KeyboardInterrupt: 

In [18]:
from collections import Counter

# 4b) After you’ve built each list, check for a majority
for idx in merged.index:
    for col in merged.columns:
        vals = [df.at[idx, col] for df in dfs]  # your list of 4 values
        cnt = Counter(vals)
        # find the most common value and its count
        most_common_val, count = cnt.most_common(1)[0]
        if count > len(vals) / 2:
            # majority found → replace with the single value
            merged.at[idx, col] = most_common_val
        else:
            # no majority → keep the full list
            merged.at[idx, col] = vals

In [22]:
adata.obs["solidity"]

0         0.8548387096774194
1         0.9558823529411764
2         0.8858447488584474
3         0.9420289855072465
4          0.935483870967742
                 ...        
299764    0.8987341772151899
299765                   1.0
299766                   nan
299767                   nan
299768                   nan
Name: solidity, Length: 299769, dtype: category
Categories (10424, object): ['0.5', '0.6', '0.7', '0.8', ..., '0.9919354838709676', '0.9932885906040269', '1.0', 'nan']

In [20]:
adata.obs

,cell_id,transcript_counts,control_probe_counts,genomic_control_counts,control_codeword_counts,unassigned_codeword_counts,deprecated_codeword_counts,total_counts,cell_area,nucleus_area,...,euler_number,extent,feret_diameter_max,intensity_max,intensity_min,intensity_mean,solidity,eccentricity,perimeter,equivalent_diameter_area
0,aaaaealn-1,68,0,0,0,0,0,68,63.083284,40.821251,...,1,0.6309523809523809,12.36931687685298,0.0,0.0,0.0,0.8548387096774194,0.8212005644969561,26.69238815542512,8.214724333230155
1,aaaaeaol-1,225,0,0,0,0,0,225,65.025002,32.828595,...,1,0.8333333333333334,13.152946437965904,0.0,0.0,0.0,0.9558823529411764,0.8916488137222875,30.485281374238575,9.09728368293446
2,aaabgafi-1,958,0,0,0,0,0,958,196.249070,92.254222,...,1,0.6807017543859649,20.615528128088304,0.0,0.0,0.0,0.8858447488584474,0.6706656434990929,55.21320343559643,15.716503163191918
3,aaabhken-1,297,0,0,1,0,1,299,68.953596,44.162814,...,1,0.6565656565656566,11.40175425099138,0.0,0.0,0.0,0.9420289855072465,0.5456934417539608,28.97056274847714,9.09728368293446
4,aaabhmik-1,33,0,0,0,0,0,33,46.556095,27.590470,...,1,0.6444444444444445,10.770329614269007,0.0,0.0,0.0,0.935483870967742,0.7768983798118462,26.97056274847714,8.593479713983122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299764,oilpooan-1,0,0,0,0,0,0,0,6.683125,6.683125,...,1,0.71,10.770329614269007,0.0,0.0,0.0,0.8987341772151899,0.3386244095312024,29.556349186104047,9.507891862878784
299765,oimajkbj-1,1,0,0,0,0,0,1,4.560781,4.560781,...,1,0.75,4.123105625617661,0.0,0.0,0.0,1.0,0.0,9.65685424949238,3.9088200952233594
299766,oimalhbk-1,23,0,0,0,0,0,23,23.165157,23.165157,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
299767,oimaomgd-1,12,0,0,0,0,0,12,59.245002,6.096094,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [ ]:
required_columns = ['label', 'area', "centroid-0","centroid-1","centroid_local-1", "centroid_weighted_local-0", "centroid_weighted_local-1"  ,"euler_number","extent","feret_diameter_max","intensity_max","intensity_min","intensity_mean","solidity",'eccentricity', 'perimeter', "equivalent_diameter_area"]
props_df_req = merged[required_columns].iloc[:, 1:]
#props_df_req = props_df_req.rename(columns=lambda c: f"0_{c}")
props_df_req.index = props_df_req.index.astype(str)


In [24]:
obs_props_df_req =  pd.concat([adata.obs, props_df_req], axis=1)
obs_props_df_req = adata.obs.join(props_df_req)
obs_props_df_req

,cell_id,transcript_counts,control_probe_counts,genomic_control_counts,control_codeword_counts,unassigned_codeword_counts,deprecated_codeword_counts,total_counts,cell_area,nucleus_area,...,0_euler_number,0_extent,0_feret_diameter_max,0_intensity_max,0_intensity_min,0_intensity_mean,0_solidity,0_eccentricity,0_perimeter,0_equivalent_diameter_area
0,aaaaealn-1,68,0,0,0,0,0,68,63.083284,40.821251,...,1,0.630952,12.369317,0.0,0.0,0.0,0.854839,0.821201,26.692388,8.214724
1,aaaaeaol-1,225,0,0,0,0,0,225,65.025002,32.828595,...,1,0.833333,13.152946,0.0,0.0,0.0,0.955882,0.891649,30.485281,9.097284
2,aaabgafi-1,958,0,0,0,0,0,958,196.249070,92.254222,...,1,0.680702,20.615528,0.0,0.0,0.0,0.885845,0.670666,55.213203,15.716503
3,aaabhken-1,297,0,0,1,0,1,299,68.953596,44.162814,...,1,0.656566,11.401754,0.0,0.0,0.0,0.942029,0.545693,28.970563,9.097284
4,aaabhmik-1,33,0,0,0,0,0,33,46.556095,27.590470,...,1,0.644444,10.77033,0.0,0.0,0.0,0.935484,0.776898,26.970563,8.59348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299764,oilpooan-1,0,0,0,0,0,0,0,6.683125,6.683125,...,1,0.71,10.77033,0.0,0.0,0.0,0.898734,0.338624,29.556349,9.507892
299765,oimajkbj-1,1,0,0,0,0,0,1,4.560781,4.560781,...,1,0.75,4.123106,0.0,0.0,0.0,1.0,0.0,9.656854,3.90882
299766,oimalhbk-1,23,0,0,0,0,0,23,23.165157,23.165157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299767,oimaomgd-1,12,0,0,0,0,0,12,59.245002,6.096094,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
obs_props_df_req.columns

Index(['cell_id', 'transcript_counts', 'control_probe_counts',
       'genomic_control_counts', 'control_codeword_counts',
       'unassigned_codeword_counts', 'deprecated_codeword_counts',
       'total_counts', 'cell_area', 'nucleus_area', 'nucleus_count',
       'segmentation_method', 'region', 'z_level', 'DAPI_PhenCyc',
       'CD8_PhenCyc', 'CD31_PhenCyc', 'CD11c_PhenCyc', 'CD34_PhenCyc',
       'CD20_PhenCyc', 'CD4_PhenCyc', 'CD21_PhenCyc', 'Ki67_PhenCyc',
       'CD68_PhenCyc', 'E_cadherin_PhenCyc', 'pten_PhenCyc', 'NF-H_PhenCyc',
       'FOXp3_PhenCyc', 'CD45_PhenCyc', 'TFAM_PhenCyc', 'PDL-1_PhenCyc',
       'SOX2_PhenCyc', 'Vimentin_PhenCyc', 'HIF1A_PhenCyc', 'CD44_PhenCyc',
       'Keratin_8_18_PhenCyc', 'aSMA_PhenCyc', 'EpCAM_PhenCyc', 'K14_PhenCyc',
       'area', 'centroid-0', 'centroid-1', 'centroid_local-1',
       'centroid_weighted_local-0', 'centroid_weighted_local-1',
       'euler_number', 'extent', 'feret_diameter_max', 'intensity_max',
       'intensity_min', 'int

In [26]:
adata.obs = obs_props_df_req

import pandas as pd

# 1. Find all object-dtype columns in obs
obj_cols = [c for c, dt in adata.obs.dtypes.items() if dt == "object"]
print("Object-dtype columns:", obj_cols)

# 2. For each such column, inspect what types you actually have
for c in obj_cols:
    sample_types = set(type(v) for v in adata.obs[c].iloc[:100])
    print(f"  {c}: {sample_types}")



# 3b. Or, if you want to keep them, convert to string:
adata.obs[obj_cols] = adata.obs[obj_cols].astype(str)

# 4. (Optional) ensure index and column names are strings
adata.obs.index   = adata.obs.index.map(str)
adata.obs.columns = adata.obs.columns.map(str)

Object-dtype columns: ['cell_id', '0_area', '0_centroid-0', '0_centroid-1', '0_centroid_local-1', '0_centroid_weighted_local-0', '0_centroid_weighted_local-1', '0_euler_number', '0_extent', '0_feret_diameter_max', '0_intensity_max', '0_intensity_min', '0_intensity_mean', '0_solidity', '0_eccentricity', '0_perimeter', '0_equivalent_diameter_area']
  cell_id: {<class 'str'>}
  0_area: {<class 'numpy.float64'>}
  0_centroid-0: {<class 'numpy.float64'>}
  0_centroid-1: {<class 'numpy.float64'>}
  0_centroid_local-1: {<class 'numpy.float64'>}
  0_centroid_weighted_local-0: {<class 'list'>}
  0_centroid_weighted_local-1: {<class 'list'>}
  0_euler_number: {<class 'numpy.int64'>}
  0_extent: {<class 'numpy.float64'>}
  0_feret_diameter_max: {<class 'numpy.float64'>}
  0_intensity_max: {<class 'numpy.float64'>}
  0_intensity_min: {<class 'numpy.float64'>}
  0_intensity_mean: {<class 'numpy.float64'>}
  0_solidity: {<class 'numpy.float64'>}
  0_eccentricity: {<class 'numpy.float64'>}
  0_perime

In [27]:
adata

AnnData object with n_obs × n_vars = 299769 × 5101
    obs: 'cell_id', 'transcript_counts', 'control_probe_counts', 'genomic_control_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'nucleus_count', 'segmentation_method', 'region', 'z_level', 'DAPI_PhenCyc', 'CD8_PhenCyc', 'CD31_PhenCyc', 'CD11c_PhenCyc', 'CD34_PhenCyc', 'CD20_PhenCyc', 'CD4_PhenCyc', 'CD21_PhenCyc', 'Ki67_PhenCyc', 'CD68_PhenCyc', 'E_cadherin_PhenCyc', 'pten_PhenCyc', 'NF-H_PhenCyc', 'FOXp3_PhenCyc', 'CD45_PhenCyc', 'TFAM_PhenCyc', 'PDL-1_PhenCyc', 'SOX2_PhenCyc', 'Vimentin_PhenCyc', 'HIF1A_PhenCyc', 'CD44_PhenCyc', 'Keratin_8_18_PhenCyc', 'aSMA_PhenCyc', 'EpCAM_PhenCyc', 'K14_PhenCyc', 'area', 'centroid-0', 'centroid-1', 'centroid_local-1', 'centroid_weighted_local-0', 'centroid_weighted_local-1', 'euler_number', 'extent', 'feret_diameter_max', 'intensity_max', 'intensity_min', 'intensity_mean', 'solidity', 'eccentricity', 'pe

In [ ]:
adata.write_h5ad(f"/Volumes/ProstateCancerEvoMain/dbs/Ongoing/{Tissue}/{TableFolder}/{Tissue}_Xenium_Phen_HE_Integrated.Xenium_Native_Table.Phen_and_ImagePropsEnriched.V2.h5ad")
#sdata.tables[adata_native] = adata
#sdata.write(f"/Volumes/ProstateCancerEvo_SpatialAnalysis/dbs/Ongoing/{Tissue}_Xenium_Phen_HE_Integrated.V4.ongoing.zarr")

In [ ]:
# SDATA is ready to go regression ! 